In [2]:
import torch
from torch.utils.data import DataLoader
import os
import pandas as pd
from utils.dataset import PandasDataset, RGB2XYZTransform
from utils.models import EfficientNet
from utils.metrics import evaluation, format_metrics
import albumentations

In [4]:
output_dimensions = 5
data_dir = '../../../dataset'
ROOT_DIR = "../.."
images_dir = os.path.join(data_dir, 'tiles')
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
df_test = pd.read_csv("../../data/test.csv")
backbone_model = 'efficientnet-b0'
pretrained_model = {
    backbone_model: f'{ROOT_DIR}/efficientnet-b0-08094119.pth'
}

In [5]:
print("Cuda", device)

Cuda cuda


In [6]:
val_transform =albumentations.Compose([
    RGB2XYZTransform()
])

In [7]:
dataloader = DataLoader(
    PandasDataset(images_dir, df_test, transforms=val_transform),
    batch_size=2,
    shuffle=False,
)

In [9]:
model = EfficientNet(
    backbone=backbone_model,
    output_dimensions=output_dimensions,
    pre_trained_model=pretrained_model
)
model.to(device)
model.load_state_dict(
    torch.load(
        f"{ROOT_DIR}/models/with-noise-xyz.pth",
        weights_only=True
    )
)

response = evaluation(model, dataloader, device)
result = format_metrics(response[0])
print(result)

Loaded pretrained weights for efficientnet-b0


100%|██████████| 796/796 [03:39<00:00,  3.62it/s]


VAL_ACC      Mean: 62.25 | Std: 1.23 | 95% CI: [60.30, 64.26]
VAL_KAPPA    Mean: 0.86 | Std: 0.01 | 95% CI: [0.85, 0.88]
VAL_F1       Mean: 0.56 | Std: 0.01 | 95% CI: [0.54, 0.58]
VAL_RECALL   Mean: 0.56 | Std: 0.01 | 95% CI: [0.54, 0.58]
VAL_PRECISION Mean: 0.56 | Std: 0.01 | 95% CI: [0.54, 0.58]
